# Module 13 Text Classfication (Naive Bayes)

- Naive Bayes Classifier 
- uses movie_reviews.csv

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("D:\\Jupyter_Notebooks\\CS654ML_DM\\Data\\movie_reviews.csv")

### How many rows?

In [3]:
print(len(df))

5000


### How many columns?

In [4]:
df.columns

Index(['text', 'label'], dtype='object')

### Show the first 10 reviews.

In [5]:
df.head(10)

,text,label
0,It's been about 14 years since Sharon Stone aw...,0
1,someone needed to make a car payment... this i...,0
2,The Guidelines state that a comment must conta...,0
3,This movie is a muddled mish-mash of clichés f...,0
4,Before Stan Laurel became the smaller half of ...,0
5,This is the best movie I've ever seen! <br /><...,1
6,The morbid Catholic writer Gerard Reve (Jeroen...,1
7,"""Semana Santa"" or ""Angel Of Death"" is a very w...",0
8,Somebody mastered the difficult task of mergin...,1
9,Why did I waste 1.5 hours of my life watching ...,0


### Sort the df by label column in decending order.

In [6]:
df.sort_values(by = 'label', ascending=False, inplace=True)

### Show the rearranged dataframe.

In [7]:
df.head()

,text,label
4999,"""The Mother"" tells of a recently widowed mid-6...",1
3983,This is a great movie to see with your girlfri...,1
2056,"Talk Radio is of course, probably not the most...",1
3998,There are some comments about this film that s...,1
3997,seriously i loved this film..i had started to ...,1


In [8]:
df.tail()

,text,label
2281,Not to be confused with the above average supe...,0
2282,"This snarky, homophobic thing was dated in 197...",0
2283,On the 26th of September 1983 a short dumpy 60...,0
2284,Although I am a fan of Heather Thomas and I ha...,0
2500,Bela Lugosi plays a doctor who will do anythin...,0


### Save the first 100 positive reviews and the last 100 negative reviews.

In [9]:
df_positive_100 = df.head(100)
#df_positive_200 = df.head(200)

In [10]:
df_negative_100 = df.tail(100)
#df_negative_200 = df.tail(200

### 100 positive reviews

In [11]:
df_positive_100

,text,label
4999,"""The Mother"" tells of a recently widowed mid-6...",1
3983,This is a great movie to see with your girlfri...,1
2056,"Talk Radio is of course, probably not the most...",1
3998,There are some comments about this film that s...,1
3997,seriously i loved this film..i had started to ...,1
...,...,...
1941,This started out to be a movie about the stree...,1
1942,"I'm out of words to describe the beauty of ""Th...",1
1943,Every once in a while in the wonderful world o...,1
4048,After watching this film I experienced a new s...,1


### 100 negative reviews

In [12]:
df_negative_100

,text,label
2198,"One of the most notorious of the banned ""Video...",0
2199,"This film holds 7.0 rating on IMDb, so even I ...",0
2204,"Saw the film at it's Lawrence, Kansas premiere...",0
2206,I have an affection for these twists on Britis...,0
2208,This is one of those movies you see in the vid...,0
...,...,...
2281,Not to be confused with the above average supe...,0
2282,"This snarky, homophobic thing was dated in 197...",0
2283,On the 26th of September 1983 a short dumpy 60...,0
2284,Although I am a fan of Heather Thomas and I ha...,0


### Save the text of the reviews into lists.

In [13]:
reviews_positive_100 = df_positive_100.iloc[:, 0].to_list()

In [14]:
reviews_negative_100 = df_negative_100.iloc[:, 0].to_list()

### Verification

In [15]:
reviews_positive_100[:10]

['"The Mother" tells of a recently widowed mid-60\'s mother of two adult children (Reid) who, on the heels of her husband\'s death, finds herself awakening from a life of sleepwalking as she has an affair with a young carpenter who is also her daughter\'s married lover. The film dwells on the quietly passive Mom, her tenuous relationship with her grown son and daughter, the silent needs she attempts to soothe in bed with her young lover, and the convolutions arising therefrom. A somewhat antiseptic drama with rumbling psychodramatic undercurrents, "The Mother" does an excellent job of dealing with uncomfortable issues realistically while avoiding gratuitous sensationalism. Will play best with more mature audiences, possibly women, who may better empathize with the central character, her needs and issues. (B+)',
 'This is a great movie to see with your girlfriend. My friend and I both love dance and ran into this movie at the video store. We had to get it. With no violence and such a wa

In [16]:
reviews_negative_100[:10]

['One of the most notorious of the banned "Video Nasties" of the 1980s is also one of the most excessively over-hyped. "Make Them Die Slowly" is about what you\'d expect from an Umberto Lenzi-directed jungle potboiler\x96inventive (yet poorly rendered) native torture techniques, some ridiculous "social commentary" (yes, even sillier than "Cannibal Holocaust"), and lots of guts being chewed. The film\'s exploitative violence, though, is often only shown in brief close-up, and never dwelt on for very long, which diminishes its effect (interpret that how you may). The dialog is Lenzi\'s usual silliness, as our male heroes show their affection for females by calling them "tw@t" and the like. The cast of familiar faces (including Lorraine De Selle, Giovanni Lombardo Radice, Zora Kerowa, and Robert Kerman) does their best in the face of the escalating idiocy (including a completely ludicrous "castration preservation"), but cannot save this overworked, lousy effort.',
 'This film holds 7.0 ra

### Load the default stop words defined in nltk.

In [17]:
from nltk.corpus import stopwords

import string

In [18]:
english_stops = stopwords.words('english')

### The default list has 179 stop words. You can add more to and remove words from this list.

In [19]:
len(english_stops)

179

### Show the first 10 stop words.

In [20]:
english_stops[:10]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]

### Define my_clean_words()

- It removes the punctuation, and normalizs every word in its lower case.
- The clean words will be returned in a list.

In [21]:
def my_clean_words(s):
    
    all_words = []
    
    review = s.translate(str.maketrans('','', string.punctuation))
    
    words = review.split()
    
    for w in words:
        w = w.lower()
        if w not in english_stops:
            all_words.append(w)
    return all_words

## Cleaning

### Clean reviews and save cleaned words for 100 positive reviews.

In [22]:
reviews_positive_100_words = []

reviews_positive_100_words_list=[]

for review in reviews_positive_100:
    
    words_cleaned = my_clean_words(review)
    
    reviews_positive_100_words_list.append(words_cleaned)
    
    for w in words_cleaned:
        reviews_positive_100_words.append(w)

### Verification

In [23]:
print(len(reviews_positive_100_words_list))

print(len(reviews_positive_100_words))

print(len(set(reviews_positive_100_words)))

100
12211
4882


### Clean reviews and save cleaned words for 100 negative reviews.

In [24]:
reviews_negative_100_words = []

reviews_negative_100_words_list=[]

for review in reviews_negative_100:
    
    words_cleaned = my_clean_words(review)
    reviews_negative_100_words_list.append(words_cleaned)
    
    for w in words_cleaned:
        reviews_negative_100_words.append(w)

In [25]:
print(len(reviews_negative_100_words_list))

print(len(reviews_negative_100_words))

print(len(set(reviews_negative_100_words)))

100
13104
4900


### All distinct words in both positive and negative reviews.

In [26]:
print(len(set(reviews_negative_100_words+reviews_positive_100_words)))

7894


### Create the feature words - attributes in words. 

- list of attribute or feature words

In [27]:
attribute_words = list(set(reviews_negative_100_words+reviews_positive_100_words))

### Verification

In [28]:
len(attribute_words)

7894

### Transform postive reviews into one-hot encodings. 

In [29]:
positive_vectors=[]

for i in range(len(reviews_positive_100_words_list)):
    
    vector = []
    
    for j in range(len(attribute_words)):
        if attribute_words[j] in reviews_positive_100_words_list[i]:
            vector.append(1)
        else:
            vector.append(0)
            
    positive_vectors.append(vector)

### Verification

In [30]:
print(len(positive_vectors))

100


### Verification

In [31]:
len(positive_vectors[0])

7894

### Transform negative reviews into one-hot encodings. 

In [32]:
negative_vectors=[]

for i in range(len(reviews_negative_100_words_list)):
    
    vector = []
    
    for j in range(len(attribute_words)):
        if attribute_words[j] in reviews_negative_100_words_list[i]:
            vector.append(1)
        else:
            vector.append(0)
   
    negative_vectors.append(vector)

### Verification

In [33]:
print(len(negative_vectors))

print(len(negative_vectors[0]))

100
7894


### Create class labels

- 1 is positive
- 0 is negative.

In [34]:
positive_negative_labels =[]

for i in range(100):
    positive_negative_labels.append(1)

for i in range(100):
    positive_negative_labels.append(0)

### Verification

In [35]:
print(len(positive_negative_labels))

200


### Convert data into numpy array.

In [36]:
import numpy as np

X = np.array(positive_vectors + negative_vectors)

### Verification

In [37]:
X.shape

(200, 7894)

### Convert labels into numpy array.

In [38]:
y = np.array(positive_negative_labels)

In [39]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0])

### Create a MultinomialNB classifier and train with X.

In [40]:
from sklearn.naive_bayes import MultinomialNB

In [41]:
nb_classifier = MultinomialNB()

nb_classifier.fit(X, y)

MultinomialNB()

### Pretend the first 10 reviews as unseen data (new data) and perform predictions.

In [42]:
nb_classifier.predict(X[:10])

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

### Pretend the last 10 reviews as unseen data (new data) and perform predictions.

In [43]:
nb_classifier.predict(X[190:])

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

### Create training and test data from the available data.

In [44]:
from sklearn.model_selection import train_test_split

from sklearn.metrics import accuracy_score

### Use half as training and half as testing.

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.5, random_state = 42)

In [46]:
NB_model = MultinomialNB()

### Create a NB classifier.

- train the classifier

In [47]:
NB_model = NB_model.fit(X_train, y_train)

### Make a prediction.

In [48]:
y_train_pred = NB_model.predict(X_train)

y_test_pred  = NB_model.predict(X_test)

### Check the accuracy of training data prediction.

In [49]:
print(accuracy_score(y_true=y_train, y_pred = y_train_pred))

1.0


### Check the accuracy of test data prediction.

In [50]:
print(accuracy_score(y_true=y_test, y_pred = y_test_pred))

0.64


### Try using cross validation.

In [51]:
from sklearn.model_selection import cross_val_score

### Use 2 folds. 
- That is, split the data into two sets (50% each). 
- One for training and one for testing.
- Swap training and test datasets for another round of performance estimation. 

In [52]:
NB_model = MultinomialNB()

scores = cross_val_score(NB_model, X, y, cv = 2)

### Check accuracies.

In [53]:
print("Cross Validation Scores: ", scores)

print("Average CV Score: ", scores.mean())

print("Number of CV Scores used in Average: ", len(scores)) 

Cross Validation Scores:  [0.75 0.71]
Average CV Score:  0.73
Number of CV Scores used in Average:  2


### This time, try 5-fold cross validation

In [54]:
print(cross_val_score(nb_classifier, X, y, cv = 5))

[0.75  0.8   0.775 0.75  0.65 ]
